In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import api_key

In [2]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,port hueneme,90,US,1601266060,82,68.00,34.15,-119.20,3.36
1,1,vardo,75,NO,1601265513,81,44.60,70.37,31.11,13.87
2,2,vaini,20,TO,1601266061,69,78.80,-21.20,-175.20,8.05
3,3,sisimiut,41,GL,1601266061,92,30.99,66.94,-53.67,7.76
4,4,roald,95,NO,1601266061,93,54.00,62.58,6.13,1.12
...,...,...,...,...,...,...,...,...,...,...
526,526,liusha,100,CN,1601266205,97,59.16,27.21,108.59,2.66
527,527,chone,90,EC,1601266205,80,72.36,-0.68,-80.10,4.85
528,528,jati,0,PK,1601266206,57,85.05,24.35,68.27,12.48
529,529,lavrentiya,94,RU,1601266206,82,35.91,65.58,-171.00,11.14


In [4]:

# Configure gmaps
gmaps.configure(api_key=api_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]



In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Temp"] >= 70) & (weather_data["Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
49,49,cururupu,0,BR,1601266072,87,75.29,-1.83,-44.87,5.44
203,203,megali vrisi,0,GR,1601266115,58,75.99,35.13,25.02,3.00
222,222,damietta,0,EG,1601266120,78,78.80,31.42,31.81,8.05
282,282,itapaci,0,BR,1601266136,34,78.46,-14.95,-49.55,1.63
358,358,boa viagem,0,BR,1601266157,85,70.38,-5.13,-39.73,3.22
372,372,dalby,0,AU,1601266161,30,71.60,-27.18,151.27,6.93
386,386,buraydah,0,SA,1601266035,41,77.00,26.33,43.98,5.82
407,407,tahta,0,EG,1601266171,33,75.20,26.77,31.50,8.05
430,430,abu zabad,0,SD,1601266177,74,75.74,12.35,29.25,4.47
431,431,lodi,0,US,1601265996,55,79.00,38.13,-121.27,3.27


In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,cururupu,BR,-1.83,-44.87,
203,megali vrisi,GR,35.13,25.02,
222,damietta,EG,31.42,31.81,
282,itapaci,BR,-14.95,-49.55,
358,boa viagem,BR,-5.13,-39.73,
372,dalby,AU,-27.18,151.27,
386,buraydah,SA,26.33,43.98,
407,tahta,EG,26.77,31.50,
430,abu zabad,SD,12.35,29.25,
431,lodi,US,38.13,-121.27,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : api_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 49: cururupu.
Missing field/result... skipping.
------------
Retrieving Results for Index 203: megali vrisi.
Missing field/result... skipping.
------------
Retrieving Results for Index 222: damietta.
Missing field/result... skipping.
------------
Retrieving Results for Index 282: itapaci.
Missing field/result... skipping.
------------
Retrieving Results for Index 358: boa viagem.
Missing field/result... skipping.
------------
Retrieving Results for Index 372: dalby.
Missing field/result... skipping.
------------
Retrieving Results for Index 386: buraydah.
Missing field/result... skipping.
------------
Retrieving Results for Index 407: tahta.
Missing field/result... skipping.
------------
Retrieving Results for Index 430: abu zabad.
Missing field/result... skipping.
------------
Retrieving Results for Index 431: lodi.
Missing field/result... skipping.
------------
Retrieving Results for Index 461: ashqelon.
Missing field/result... skipping.
------------
Retr

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,cururupu,BR,-1.83,-44.87,
203,megali vrisi,GR,35.13,25.02,
222,damietta,EG,31.42,31.81,
282,itapaci,BR,-14.95,-49.55,
358,boa viagem,BR,-5.13,-39.73,
372,dalby,AU,-27.18,151.27,
386,buraydah,SA,26.33,43.98,
407,tahta,EG,26.77,31.50,
430,abu zabad,SD,12.35,29.25,
431,lodi,US,38.13,-121.27,
